<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/802%20code/bart-base-paradetox-1Token-split-masked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] datasets
!pip install transformers[torch]
!pip install sacrebleu sentencepiece
!pip install evaluate
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import BartForConditionalGeneration

# Load model directly
from transformers import AutoTokenizer, AutoModel

raw_datasets = load_dataset("HamdanXI/paradetox-1Token-Split")
checkpoint = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = BartForConditionalGeneration.from_pretrained(checkpoint)

raw_datasets = raw_datasets.rename_column("en_toxic_comment", "input")
raw_datasets = raw_datasets.rename_column("en_neutral_comment", "labels")
raw_datasets = raw_datasets.remove_columns("edit_ops")
raw_datasets = raw_datasets.remove_columns("masked_comment")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3784 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/811 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/811 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['input', 'labels'],
        num_rows: 3784
    })
    validation: Dataset({
        features: ['input', 'labels'],
        num_rows: 811
    })
    test: Dataset({
        features: ['input', 'labels'],
        num_rows: 811
    })
})

In [5]:
max_token_length_input = max(len(tokenizer.encode(item)) for item in raw_datasets["train"]["input"])
max_token_length_label = max(len(tokenizer.encode(item)) for item in raw_datasets["train"]["labels"])

print(f"Max token length for input: {max_token_length_input}")
print(f"Max token length for label: {max_token_length_label}")

if max_token_length_input > max_token_length_label:
    highest_length = max_token_length_input
else:
    highest_length = max_token_length_label

print(highest_length)

Max token length for input: 29
Max token length for label: 29
29


In [6]:
def tokenize_function(example):
    source = example["input"]
    target = example["labels"]

    # Tokenizing source and target without returning tensors and without padding
    tokenized_source = tokenizer(source, truncation=True, max_length=highest_length, return_tensors="pt")
    tokenized_target = tokenizer(target, truncation=True, max_length=highest_length, return_tensors="pt")

    return {
        "input_ids": tokenized_source["input_ids"][0],
        "attention_mask": tokenized_source["attention_mask"][0],
        "labels": tokenized_target["input_ids"][0]
    }

tokenized_datasets = raw_datasets.map(tokenize_function)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.1)
# tokenized_datasets["validation"] = tokenized_datasets.pop("test")

Map:   0%|          | 0/3784 [00:00<?, ? examples/s]

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

In [7]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(output_dir="bart-base-paradetox-1Token-split-masked", push_to_hub=True)
training_args = training_args.set_save(steps=10000)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [8]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.394900
1000,0.232700


TrainOutput(global_step=1419, training_loss=0.2688363191525309, metrics={'train_runtime': 853.5619, 'train_samples_per_second': 13.3, 'train_steps_per_second': 1.662, 'total_flos': 143440501800960.0, 'train_loss': 0.2688363191525309, 'epoch': 3.0})

In [9]:
trainer.push_to_hub()

events.out.tfevents.1700720762.730d23365a23.5907.0:   0%|          | 0.00/6.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

'https://huggingface.co/HamdanXI/bart-base-paradetox-1Token-split-masked/tree/main/'